In [17]:
# Neural Network for binary classification and regression tasks

In [18]:
import numpy as np

In [19]:
# with sigmoid as activation (you can remove sigmoid to perform regression tasks)
# if you want to perform multi-label classification fuck you
class NeuralNetwork:
    def __init__(self, no_of_layers):
        self.num_layers = no_of_layers
        num_neurons = []
        hidden_count = 1
        for i in range(no_of_layers):
            if i != no_of_layers - 1:
                print("Enter Number of Neurons for Hidden Layer "+str(hidden_count))
                num_neurons.append(int(input()))
                hidden_count += 1
            else:
                print("Number of neurons in output layer = 1")
                num_neurons.append(1)
                
            
        self.numb_neurons = num_neurons
        self.weights = []
        self.bias = []
        self.a = []
        self.dz = []
        self.delta_w = []
        self.delta_b = []
        self.y_pred = None
        self.trained = False
        self.lr = 0
        self.m = 0
    def train(self, train_x, train_y, epochs, lr):
        # weight and bias initializations
        if self.trained == False:
            for i in range(self.num_layers):
                if i == 0:
                   self.weights.append(np.random.randn(self.numb_neurons[i],train_x.shape[1]))
                else:
                   self.weights.append(np.random.randn(self.numb_neurons[i],self.numb_neurons[i - 1]))
                self.bias.append(np.random.randn(self.numb_neurons[i],1))
                self.y_pred = np.zeros((1,len(train_y)))
                self.lr = lr
                self.m = len(train_x)
        self.trained = True
        for i in range(epochs):    
            self.forward_pass(train_x)
            self.backward_pass(train_x, train_y)
            y_t = train_y.reshape(1, len(train_y))
            print(self.loss_func(y_t, self.y_pred))

    def test(self, x_test):
        a_ = []
        
        for i in range(self.num_layers):
            if i == 0:
                z = np.matmul(self.weights[i], x_test.T) + self.bias[i]
                a = 1/(1+np.exp(-z))
                a_.append(a)
            else:
                z = np.matmul(self.weights[i], a_[i-1]) + self.bias[i]
                a = 1/(1+np.exp(-z))
                a_.append(a)
            y_p = a[-1].flatten()
            # only for classification tasks
            for i in range(len(y_p)):
                if y_p[i] >= 0.5:
                   y_p[i] = 1
                else:
                   y_p[i] = 0
        return y_p

    
    def loss_func(self, y_true, y_pred):
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred)) 
    
    def forward_pass(self, train_x):
        for i in range(self.num_layers):
            if i == 0:
                z = np.matmul(self.weights[i], train_x.T) + self.bias[i]
                a = 1/(1+np.exp(-z)) # remove this to perform regression tasks
                self.a.append(a)
            else:
                z = np.matmul(self.weights[i], self.a[i-1]) + self.bias[i]
                a = 1/(1+np.exp(-z))
                self.a.append(a)

    def backward_pass(self, train_x, train_y):
        for i in range(self.num_layers - 1, -1,-1):
            # last layer case where derivative of loss is needed
            if i == self.num_layers - 1:
                dz = self.a[i] - train_y
                self.dz.insert(0, dz)
                delta_w = 1/self.m * np.matmul(dz ,self.a[i-1].T)  
                self.delta_w.insert(0, delta_w)
                self.delta_b.insert(0, 1/self.m * np.sum(dz))
            # first layer case where a[i-1] is actually train_x
            elif i == 0:               
                dz = np.matmul(self.weights[i+1].T, self.dz[0]) * (self.a[i] * (1 - self.a[i]))
                self.dz.insert(0, dz)
                delta_w = 1/self.m * np.matmul(dz, train_x)
                self.delta_w.insert(0, delta_w)
                self.delta_b.insert(0, 1/self.m * np.sum(dz))
            # all hidden layer cases
            else:
                dz = np.matmul(self.weights[i+1].T, self.dz[0]) * (self.a[i] * (1 - self.a[i]))
                self.dz.insert(0, dz)
                delta_w = 1/self.m * np.matmul(dz, self.a[i-1].T)
                self.delta_w.insert(0, delta_w)
                self.delta_b.insert(0, 1/self.m * np.sum(dz))
        # updating weights
        for i in range(len(self.weights)):
            self.weights[i] -= self.lr * self.delta_w[i]
            self.bias[i] -= self.lr * np.broadcast_to(self.delta_b[i],self.bias[i].shape)
        self.y_pred = self.a[-1]
        self.dz, self.a, self.delta_w, self.delta_b  = [],[],[],[]                

In [ ]:
nn = NeuralNetwork(6)

Enter Number of Neurons for Hidden Layer 1


In [14]:
# clean and normalize your data here
train_x = np.array([[0,0],[0,1],[1,0],[1,1]])
train_y = np.array([0,0,0,1])

In [15]:
# train_x should be scaled and of shape (training examples, features) i.e (batch_size, input)
# train_y should be a one dimensional vector
# both train_x and train_y must be numpy arrays
# 3rd param is no. of epochs
# 4th param is learning rate
nn.train(train_x,train_y,1000, 0.1)

0.9760476930722307
0.8798412356433357
0.8063399699240392
0.7502441188233093
0.7073162584036511
0.6742902030281461
0.6487026336895259
0.6287167389493371
0.6129686208779118
0.6004449733050521
0.5903902449783088
0.5822379932570189
0.5755607845160009
0.5700338160189564
0.5654085033816951
0.5614932448164658
0.5581393417782254
0.5552306300325809
0.5526757911827409
0.5504026111547176
0.548353661851035
0.5464830302327437
0.54475382375418
0.5431362553328334
0.5416061639833497
0.5401438652169448
0.5387332527139652
0.5373610926896877
0.5360164669389721
0.5346903312720598
0.533375164006343
0.5320646851134779
0.5307536310781524
0.5294375738927304
0.5281127751721407
0.5267760683306445
0.5254247632666035
0.5240565691639214
0.5226695319215647
0.5212619834268785
0.5198325004403751
0.5183798712940668
0.5169030689486919
0.5154012292274945
0.5138736332611025
0.5123196933514108
0.5107389416015472
0.5091310207711706
0.5074956769072092
0.5058327533741536
0.5041421859686712
0.502423998853421
0.500678301086722

In [16]:
nn.test(np.array([[0,0],[0,1],[1,0],[1,1]]))

array([0., 0., 0., 1.])